In [2]:
import tensorflow_datasets as tfds 

imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

In [3]:
train_data, test_data = imdb['train'], imdb['test']

In [4]:
import numpy as np 

training_sentences = []
training_labels = []

testing_sentences = [] 
testing_labels = []


In [5]:
for s,l in train_data:
    training_sentences.append(str(s.numpy().decode('utf8')))
    training_labels.append(l.numpy())

In [6]:
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [7]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [8]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<!@#>"

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [11]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [12]:
import tensorflow as tf 

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [16]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data=(testing_padded,testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4979 - accuracy: 0.7356 - val_loss: 0.3448 - val_accuracy: 0.8508
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2388 - accuracy: 0.9079 - val_loss: 0.3647 - val_accuracy: 0.8421
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0919 - accuracy: 0.9757 - val_loss: 0.4499 - val_accuracy: 0.8301
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0222 - accuracy: 0.9969 - val_loss: 0.5258 - val_accuracy: 0.8291
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0059 - accuracy: 0.9994 - val_loss: 0.5916 - val_accuracy: 0.8278
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6415 - val_accuracy: 0.8284
Epoch 7/10
782/782 [==============================] - 4s 4ms/step - loss: 8.3874e-04 - accuracy: 1.0000 - val_loss: 0.6825 - val_accuracy: 0.8298
Ep

In [20]:
model1.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data=(testing_padded,testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 4s 6ms/step - loss: 0.5728 - accuracy: 0.7216 - val_loss: 0.4021 - val_accuracy: 0.8410
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3411 - accuracy: 0.8588 - val_loss: 0.3390 - val_accuracy: 0.8531
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2737 - accuracy: 0.8880 - val_loss: 0.3304 - val_accuracy: 0.8574
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2377 - accuracy: 0.9071 - val_loss: 0.3430 - val_accuracy: 0.8542
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2115 - accuracy: 0.9204 - val_loss: 0.3587 - val_accuracy: 0.8511
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1901 - accuracy: 0.9298 - val_loss: 0.3832 - val_accuracy: 0.8447
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1727 - accuracy: 0.9390 - val_loss: 0.4114 - val_accuracy: 0.8398
Epoch 

In [21]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <!@#> into a big arm chair and <!@#> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <!@#> to cross no dangerous waters just a warm and witty <!@#> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [22]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


In [23]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


LSTM VERSION

In [24]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [25]:
lstm_model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
lstm_model.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data=(testing_padded,testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 71s 91ms/step - loss: 0.4649 - accuracy: 0.7699 - val_loss: 0.3493 - val_accuracy: 0.8522
Epoch 2/10
782/782 [==============================] - 73s 93ms/step - loss: 0.2865 - accuracy: 0.8846 - val_loss: 0.3490 - val_accuracy: 0.8483
Epoch 3/10
782/782 [==============================] - 81s 103ms/step - loss: 0.2229 - accuracy: 0.9135 - val_loss: 0.3736 - val_accuracy: 0.8424
Epoch 4/10
782/782 [==============================] - 84s 108ms/step - loss: 0.1718 - accuracy: 0.9371 - val_loss: 0.5347 - val_accuracy: 0.8238
Epoch 5/10
782/782 [==============================] - 92s 118ms/step - loss: 0.1303 - accuracy: 0.9531 - val_loss: 0.5984 - val_accuracy: 0.8249
Epoch 6/10
782/782 [==============================] - 85s 109ms/step - loss: 0.0903 - accuracy: 0.9688 - val_loss: 0.6180 - val_accuracy: 0.8205
Epoch 7/10
782/782 [==============================] - 76s 97ms/step - loss: 0.0605 - accuracy: 0.9803 - val_loss: 0.6320 - val_accur

In [27]:
gru_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [28]:
gru_model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
gru_model.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data=(testing_padded,testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 65s 83ms/step - loss: 0.5064 - accuracy: 0.7308 - val_loss: 0.3550 - val_accuracy: 0.8472
Epoch 2/10
782/782 [==============================] - 64s 81ms/step - loss: 0.3009 - accuracy: 0.8782 - val_loss: 0.3528 - val_accuracy: 0.8466
Epoch 3/10
782/782 [==============================] - 64s 81ms/step - loss: 0.2267 - accuracy: 0.9119 - val_loss: 0.3935 - val_accuracy: 0.8451
Epoch 4/10
782/782 [==============================] - 64s 82ms/step - loss: 0.1628 - accuracy: 0.9418 - val_loss: 0.4393 - val_accuracy: 0.8339
Epoch 5/10
782/782 [==============================] - 64s 82ms/step - loss: 0.1154 - accuracy: 0.9622 - val_loss: 0.5322 - val_accuracy: 0.8246
Epoch 6/10
782/782 [==============================] - 66s 85ms/step - loss: 0.0755 - accuracy: 0.9761 - val_loss: 0.5771 - val_accuracy: 0.8124
Epoch 7/10
782/782 [==============================] - 79s 101ms/step - loss: 0.0558 - accuracy: 0.9814 - val_loss: 0.7519 - val_accuracy

In [30]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
conv_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
conv_model.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data=(testing_padded,testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4415 - accuracy: 0.7837 - val_loss: 0.3473 - val_accuracy: 0.8505
Epoch 2/10
782/782 [==============================] - 12s 16ms/step - loss: 0.2774 - accuracy: 0.8860 - val_loss: 0.3564 - val_accuracy: 0.8456
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2233 - accuracy: 0.9134 - val_loss: 0.3911 - val_accuracy: 0.8376
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.1815 - accuracy: 0.9336 - val_loss: 0.4188 - val_accuracy: 0.8336
Epoch 5/10
782/782 [==============================] - 12s 16ms/step - loss: 0.1454 - accuracy: 0.9498 - val_loss: 0.4956 - val_accuracy: 0.8266
Epoch 6/10
782/782 [==============================] - 13s 17ms/step - loss: 0.1154 - accuracy: 0.9630 - val_loss: 0.5422 - val_accuracy: 0.8187
Epoch 7/10
782/782 [==============================] - 13s 16ms/step - loss: 0.0894 - accuracy: 0.9726 - val_loss: 0.6109 - val_accuracy: